<a href="https://colab.research.google.com/github/yuvalmay30/waldo-finder/blob/main/Waldo_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-174-gc4cb7c6 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.1/78.2 GB disk)


In [3]:
# %cd yolov5
import utils
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob


%matplotlib inline

# Data Importing

In [4]:
%rm -r ../datasets
%rm -r ../waldo-finder

rm: cannot remove '../datasets': No such file or directory
rm: cannot remove '../waldo-finder': No such file or directory


In [5]:
%cd ..
!git clone https://github.com/yuvalmay30/waldo-finder.git  # clone
%cd yolov5/

/content
Cloning into 'waldo-finder'...
remote: Enumerating objects: 4769, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 4769 (delta 11), reused 6 (delta 1), pack-reused 4744
Receiving objects: 100% (4769/4769), 923.62 MiB | 26.81 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Checking out files: 100% (3093/3093), done.
/content/yolov5


In [6]:
%cp -R ../waldo-finder/datasets ../datasets
%cp -R ../drive/MyDrive/CV\ Projects/Waldo\ Finder/last_yolov5x_200epochs_freeze.pt ../yolov5/models/

In [7]:
import shutil

def copy_dataset_yaml_to_yolo_data_directory(dataset_dir_name):
  source_file = f'../datasets/{dataset_dir_name}/{dataset_dir_name}.yaml'
  destination_file = f'./data/{dataset_dir_name}.yaml'
  
  shutil.copy(source_file, destination_file)


copy_dataset_yaml_to_yolo_data_directory('source_dataset_v2')
copy_dataset_yaml_to_yolo_data_directory('source_dataset_v2_cropped_val')
copy_dataset_yaml_to_yolo_data_directory('source_dataset_v2_tiled_5_4')

## Weights & Biases

In [8]:
%pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 41.7 MB/s 
     |████████████████████████████████| 181 kB 79.9 MB/s 
     |████████████████████████████████| 144 kB 77.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Training

In [ ]:
# !python train.py --batch 16 --epochs 10 --data source_dataset_v2.yaml --weights yolov5x.pt --cache --freeze 10
!python train.py --batch 16 --epochs 10 --data source_dataset_v2.yaml --weights yolov5s.pt --cache

wandb: Currently logged in as: yuvalmay30 (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=source_dataset_v2.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/ultralytics/yolov5
   c65f990..e062169  pre-commit-ci-update-config -> origin/pre-comm

## Utils functions for predictions

In [18]:
from typing import Tuple
import cv2
import os
import math
import pdb
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
from pathlib import Path

# %load ../yolov5/utils/general.py
# from utils.general import xywhn2xyxy


def xywhn2xyxyn(xywhn):
  xyxyn = xywhn.clone()

  xyxyn[:, 0] = xywhn[:, 0] - xywhn[:, 2] / 2
  xyxyn[:, 1] = xywhn[:, 1] - xywhn[:, 3] / 2
  xyxyn[:, 2] = xywhn[:, 0] + xywhn[:, 2] / 2
  xyxyn[:, 3] = xywhn[:, 1] + xywhn[:, 3] / 2

  return xyxyn


def sort_predictions_by_confidence(predictions: pd.DataFrame):
  return predictions.sort_values(by=['confidence'], ascending=False)


def convert_predictions_bboxes_values_to_int(predictions):
  return predictions.astype({'xmin': 'int', 'ymin': 'int', 'xmax': 'int', 'ymax': 'int'})


def get_top_predictions(predictions, top=5, convert_to_int=True):
  sorted_predictions = sort_predictions_by_confidence(predictions)
  top_predictions = sorted_predictions[:top]
  top_predictions = top_predictions[['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']]
  top_predictions = top_predictions.astype({'class': 'int'})

  if convert_to_int:
    top_predictions = convert_predictions_bboxes_values_to_int(top_predictions)
    top_predictions = top_predictions.to_numpy()

  return top_predictions


def get_normalized_top_predictions_from_model_results(results, top=5, convert_to_int=True):
  predictions = results.pandas().xyxyn[0]
  top_predictions = get_top_predictions(predictions, top, convert_to_int)
  
  top_predictions = torch.tensor(top_predictions.values)
  return top_predictions


def get_denormalized_top_predictions_from_model_results(results, top=5, convert_to_int=True):
  predictions = results.pandas().xyxy[0]
  top_predictions = get_top_predictions(predictions, top, convert_to_int)

  # top_predictions = torch.tensor(top_predictions.values)
  return top_predictions


def draw_prediction_bbox(image, top_left_coordinates: Tuple, bottom_right_coordinates: Tuple):
  red_color = (0, 0, 255)
  thickness = 6
  
  cv2.rectangle(image, top_left_coordinates, bottom_right_coordinates, red_color, thickness)
  return image


def draw_text_on_image(image, confidence, location):
  text = 'Waldo ' + str(confidence)
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 0.9
  red_color = (0, 0, 255)
  thickness = 2
  
  cv2.putText(image, text, location, font, font_scale, red_color, thickness)
  return image


def draw_predictions_on_image(image, predictions):
  for prediction in predictions:
    x1, y1, x2, y2, confidence, _ = prediction
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    image = draw_prediction_bbox(image, (x1, y1), (x2, y2))
    image = draw_text_on_image(image, confidence, (x1, y1))
  
  return image


def change_file_suffix_to_txt(filename: str) -> str:
  filename = filename[:-3] + 'txt'
  return filename


def load_labels(labels_dir_path: str, filename: str):
  labels_filename = change_file_suffix_to_txt(filename)
  labels_file_path = Path(labels_dir_path) / labels_filename

  with open(labels_file_path) as labels_file:
    labels_as_strings = labels_file.readlines()
  
  labels = torch.tensor([])
  for label in labels_as_strings:
    data = label.split()
    data = [float(element) for element in data]
    data = torch.tensor(data)
    
    labels = torch.cat((labels, data))
  
  if labels.dim() == 1:
    labels = labels.unsqueeze(dim=0)

  return labels


def draw_label_bbox(image, top_left_coordinates: Tuple, bottom_right_coordinates: Tuple):
  green_color = (0, 255, 0)
  thickness = 4
  
  cv2.rectangle(image, top_left_coordinates, bottom_right_coordinates, green_color, thickness)
  return image


def extract_bbox_from_label(label, img_width, img_height): 
  w, h = img_width, img_height

  bbox = label[1:]
  x, y, w, h = bbox * torch.tensor([w, h, w, h])
  x1, x2 = int(x)-int(w/2), int(x)+int(w/2)
  y1, y2 = int(y)-int(h/2), int(y)+int(h/2)

  return x1, y1, x2, y2


def draw_labels_on_image(image, labels):
  h, w = image.shape[:2]

  for label in labels:
    xmin, ymin, xmax, ymax = extract_bbox_from_label(label, w, h)
    image = draw_label_bbox(image, (xmin, ymin), (xmax, ymax))

  return image


def get_image_from_model_results(results):
  img = results.imgs[0]
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img 

# Validation

In [9]:
def load_model():
  model = torch.hub.load('./', 'custom', source='local', path='./models/last_yolov5x_200epochs_freeze.pt', force_reload=True)
  model.conf = 0.00001
  return model

Pseudo-code:

1. Load model
2. Run detection on an image
3. Get top 5 predicitons
4. Calculate IOU with the grouth truth  
  a. Use `process_batch` function from yolov5  
  b. Set the top normalized detections (xmin, ymin, xmax, ymax, confidence, class)  
  c. Set the normalized labels
5. Keep the prediction with IOU > 0.1
6. If there are any predictions left, mark this image as `hit`
7. Calculate mean

In [10]:
%rm -r ./runs/error_analysis/
%mkdir runs
%mkdir runs/error_analysis

rm: cannot remove './runs/error_analysis/': No such file or directory


In [27]:
%load ../yolov5/val.py
from val import process_batch


def validate(images_path, labels_dir_path):
  error_analysis_path = Path('./runs') / 'error_analysis'
  hits = np.array([])
  iou_threshold = torch.tensor([0.1])
  
  model = load_model()

  for dir_entry in os.scandir(images_path):
    image_path = dir_entry.path
    results = model(image_path, size=2048)

    top_predictions = get_normalized_top_predictions_from_model_results(results, convert_to_int=False)
    
    labels_file_name = dir_entry.name
    labels = load_labels(labels_dir_path, labels_file_name)
    labels_bbox_as_xywhn = labels[:, 1:]

    labels_bbox_as_xyxyn = xywhn2xyxyn(labels_bbox_as_xywhn)
    labels[:, 1:] = labels_bbox_as_xyxyn

    true_positives = process_batch(top_predictions, labels, iou_threshold)

    is_hit = torch.any(true_positives)
    hits = np.append(hits, is_hit)

    if not is_hit:
      img_with_drawn_predictions_and_labels = draw_labels_and_top_predictions_for_single_image(results, labels_dir_path, labels_file_name)
      save_file_path = error_analysis_path / labels_file_name
      cv2.imwrite(save_file_path, img_with_drawn_predictions_and_labels)


  hit_rate = hits.mean()
  print(f'The hit-rate is: {hit_rate}')


images_path = '../datasets/source_dataset_v2_cropped_val/valid/images'
labels_path = '../datasets/source_dataset_v2_cropped_val/valid/labels'

validate(images_path, labels_path)  

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v6.1-174-gc4cb7c6 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


The hit-rate is: 0.9


In [28]:
test_images_path = '../datasets/source_dataset_v2_cropped_val/test/images'
test_labels_path = '../datasets/source_dataset_v2_cropped_val/test/labels'

validate(test_images_path, test_labels_path)  

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v6.1-174-gc4cb7c6 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


RuntimeError: ignored

In [ ]:
# !python val.py --weights models/last_yolov5x_200epochs_freeze.pt --data source_dataset_v2_cropped_val.yaml --half --conf-thres 0.001 --iou 0.1 --img 2048 --single-cls
# !python val.py --weights models/last_yolov5x_200epochs_freeze.pt --data source_dataset_v2.yaml --half --conf-thres 0.01 --img 2048
# !python detect.py --weights ./models/last_yolov5x_200epochs_freeze.pt --source ../datasets/source_dataset_v2_cropped_val/valid/images --conf-thres 0.01 --iou 0.45 --img 2048

## Visualization

In [12]:
%rm -r ./runs/comparison/
%mkdir runs
%mkdir runs/comparison

rm: cannot remove './runs/comparison/': No such file or directory
mkdir: cannot create directory ‘runs’: File exists


In [26]:
def draw_labels_and_top_predictions_for_single_image(results, labels_dir_path, labels_file_name):
  top_predictions = get_denormalized_top_predictions_from_model_results(results)

  img = get_image_from_model_results(results)
  img_with_drawn_predictions = draw_predictions_on_image(img, top_predictions)

  labels = load_labels(labels_dir_path, labels_file_name)
  img_with_drawn_predictions_and_labels = draw_labels_on_image(img_with_drawn_predictions, labels)
  
  return img_with_drawn_predictions_and_labels

In [20]:
def draw_labels_and_top_predictions(images_dir_path, labels_dir_path):
  model = load_model()

  for dir_entry in os.scandir(images_dir_path):
    labels_file_name = dir_entry.name

    results = model(dir_entry.path, size=2048)
    img_with_drawn_predictions_and_labels = draw_labels_and_top_predictions_for_single_image(results, labels_dir_path, labels_file_name)
    
    file_path = Path('./runs') / 'comparison' / labels_file_name
    cv2.imwrite(file_path, img_with_drawn_predictions_and_labels)
  
  print('Results saved to: /runs/comparison/')


images_dir_path = '../datasets/source_dataset_v2_cropped_val/valid/images'
labels_dir_path = '../datasets/source_dataset_v2_cropped_val/valid/labels'

draw_labels_and_top_predictions(images_dir_path, labels_dir_path)


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v6.1-174-gc4cb7c6 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


Results saved to: /runs/comparison/
